# 0-1. 남성 월별 거주인구 -> 일별 거주인구

- 거주인구데이터가 월별로 되어있기 때문에 에측에 사용하기 위해 일별 변환
- 정규분포를 따르는 난수추출 방식을 사용

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import time

In [2]:
Data_5 = pd.read_csv('../data/train_data/03_거주인구_RESIDENT_POP.CSV',encoding='cp949')

In [3]:
Data_5

,base_year,base_month,city,emd_cd,emd_nm,sex,resid_reg_pop,foreign_pop,total_pop
0,2018,1,제주시,50110590,건입동,남성,5085,146,5231
1,2018,1,제주시,50110590,건입동,여성,4715,82,4797
2,2018,1,제주시,50110256,구좌읍,남성,7965,368,8333
3,2018,1,제주시,50110256,구좌읍,여성,7609,184,7793
4,2018,1,서귀포시,50130253,남원읍,남성,9806,428,10234
...,...,...,...,...,...,...,...,...,...
3607,2021,6,제주시,50110250,한림읍,여성,10341,1140,11481
3608,2021,6,제주시,50110600,화북동,남성,12062,169,12231
3609,2021,6,제주시,50110600,화북동,여성,12238,161,12399
3610,2021,6,서귀포시,50130550,효돈동,남성,2660,39,2699


In [4]:
Data_5 = Data_5.astype({'base_year': 'str', 'base_month' : 'str'})
Data_5['base_date'] = Data_5['base_year'] + '-' + Data_5['base_month']

In [5]:
Data_5['base_date'] = Data_5['base_date'].apply(lambda x: x.replace('-','-0') if len(x)==6 else x)

In [6]:
Data_5 = Data_5.drop(['city','emd_cd','base_year','base_month'],axis=1)

In [7]:
Data_5 = Data_5.groupby(['base_date','emd_nm','sex']).sum().reset_index()

In [8]:
Data_5 = Data_5[Data_5['sex']=='남성']
Data_5.drop('sex',axis=1,inplace=True)

In [9]:
Data_5 = Data_5.drop(Data_5[Data_5.emd_nm == '우도면'].index)

In [10]:
Data_5 = Data_5.drop(Data_5[Data_5.emd_nm == '추자면'].index)

In [11]:
Date = pd.date_range('2018-01-01','2021-06-30',freq="D")

In [12]:
emd = Data_5.emd_nm.unique()

In [13]:
Date_df = pd.DataFrame(Date)
Date_df.columns = ['Date']

In [14]:
emd_df = pd.DataFrame(emd)
emd_df.columns = ['emd_nm']

In [15]:
Date_new = []
for i in range(len(Date)):
    Date_new.append(str(Date[i]))

In [16]:
Data_renew = []
for i in range(len(Date_new)):
    Data_renew.append(Date_new[i][:10])

In [17]:
import itertools
result = itertools.product(Data_renew, emd)
result = list(result)

In [18]:
len(result)

52357

In [19]:
len(emd)

41

In [20]:
len(Data_renew)

1277

In [21]:
df = pd.DataFrame(list(result))
df.columns = ['base_date','emd_nm']

In [22]:
df = df.sort_values(by=['emd_nm','base_date'])
df = df.reset_index()
df = df.drop('index',axis=1)

In [23]:
Data_5 = Data_5.sort_values(by=['emd_nm','base_date'])
Data_5 = Data_5.reset_index()
Data_5 = Data_5.drop('index',axis=1)

In [24]:
size = [31,28,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30,31,31,30,31,30,31,31,29,31,30,31,30,31,31,30,31,30,31,31,28,31,30,31,30]

In [25]:
size = size*41

In [26]:
resid_pop = []
for i in range(len(Data_5)):
    a=list(np.random.normal(Data_5['resid_reg_pop'][i], 50,size=size[i]).astype('int'))
    resid_pop.extend(a)

In [27]:
df['resid_pop'] = resid_pop

In [28]:
foreign_pop = []
for i in range(len(Data_5)):
    a=list(np.random.normal(Data_5['foreign_pop'][i], size=size[i]).astype('int'))
    foreign_pop.extend(a)

In [29]:
df['foreign_pop'] = foreign_pop

In [30]:
df['total_pop'] = df['resid_pop'] + df['foreign_pop']

In [31]:
df

,base_date,emd_nm,resid_pop,foreign_pop,total_pop
0,2018-01-01,건입동,5161,146,5307
1,2018-01-02,건입동,5013,145,5158
2,2018-01-03,건입동,5220,146,5366
3,2018-01-04,건입동,5082,146,5228
4,2018-01-05,건입동,5108,146,5254
...,...,...,...,...,...
52352,2021-06-26,효돈동,2685,39,2724
52353,2021-06-27,효돈동,2677,38,2715
52354,2021-06-28,효돈동,2790,40,2830
52355,2021-06-29,효돈동,2621,38,2659


## 0-1-1. emd_nm 의 '알수없음'은 해당일 전체 읍면동 거주인구를 평균을 냄

In [32]:
unknown = df.groupby('base_date').mean().reset_index()

In [33]:
unknown['emd_nm'] = '알수없음'

In [34]:
unknown['resid_pop'] = unknown['resid_pop'].astype('int')

In [35]:
unknown['foreign_pop'] = unknown['foreign_pop'].astype('int')

In [36]:
unknown['total_pop'] = unknown['total_pop'].astype('int')

In [37]:
unknown

,base_date,resid_pop,foreign_pop,total_pop,emd_nm
0,2018-01-01,8016,282,8299,알수없음
1,2018-01-02,8031,282,8314,알수없음
2,2018-01-03,8036,282,8319,알수없음
3,2018-01-04,8040,282,8323,알수없음
4,2018-01-05,8016,282,8299,알수없음
...,...,...,...,...,...
1272,2021-06-26,8217,288,8506,알수없음
1273,2021-06-27,8229,288,8517,알수없음
1274,2021-06-28,8221,288,8510,알수없음
1275,2021-06-29,8217,288,8506,알수없음


In [38]:
df_final = pd.concat([df,unknown])

In [39]:
df_final

,base_date,emd_nm,resid_pop,foreign_pop,total_pop
0,2018-01-01,건입동,5161,146,5307
1,2018-01-02,건입동,5013,145,5158
2,2018-01-03,건입동,5220,146,5366
3,2018-01-04,건입동,5082,146,5228
4,2018-01-05,건입동,5108,146,5254
...,...,...,...,...,...
1272,2021-06-26,알수없음,8217,288,8506
1273,2021-06-27,알수없음,8229,288,8517
1274,2021-06-28,알수없음,8221,288,8510
1275,2021-06-29,알수없음,8217,288,8506


In [40]:
df_final.columns = ['일자', '읍면동', '남_내거주', '남_외거주', '남_총거주']

In [42]:
df_final

,일자,읍면동,남_내거주,남_외거주,남_총거주
0,2018-01-01,건입동,5161,146,5307
1,2018-01-02,건입동,5013,145,5158
2,2018-01-03,건입동,5220,146,5366
3,2018-01-04,건입동,5082,146,5228
4,2018-01-05,건입동,5108,146,5254
...,...,...,...,...,...
1272,2021-06-26,알수없음,8217,288,8506
1273,2021-06-27,알수없음,8229,288,8517
1274,2021-06-28,알수없음,8221,288,8510
1275,2021-06-29,알수없음,8217,288,8506


In [41]:
df_final.to_csv('../data/processed_data/남성_거주인구.csv',encoding='cp949',index=False)